# Loading the parsed values from the csv file

In [1]:
import sqlite3

db_path = "../data/artefacts/parsed.db"
db = sqlite3.connect(db_path, isolation_level="EXCLUSIVE")
cur = db.cursor()

In the `parsed` table, the pattern is :
```sqlite
domain TEXT,
mode TEXT,
scan_depth INTEGER,
retrieved_date TIMESTAMP,
registrar TEXT,
iana_id INTEGER,
creation_date TIMESTAMP,
expiration_date TIMESTAMP,
nameservers TEXT,
emails TEXT
```

In [2]:
cur.execute("CREATE TABLE IF NOT EXISTS parsed( "
            "domain TEXT,"
            "mode TEXT,"
            "scan_depth INTEGER,"
            "retrieved_date TIMESTAMP DEFAULT 0,"
            "registrar TEXT,"
            "iana_id INTEGER DEFAULT 0,"
            "creation_date TIMESTAMP DEFAULT 0,"
            "expiration_date TIMESTAMP DEFAULT 0,"
            "nameservers TEXT,"
            "emails TEXT)")
db.commit()

In [3]:
import csv

with open("../data/artefacts/data.csv") as f:
    entries = csv.DictReader(f)
    cur.executemany("INSERT INTO parsed VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                   ((e['domain'], e['mode'], e['scan_depth'], e['retrieved_date'], 
                     e['registrar'], e['iana_id'], e['creation_date'], e['expiration_date'],
                    e['nameservers'], e['emails'])
                     for e in entries))

cur.execute("CREATE INDEX IF NOT EXISTS parsed_domain ON parsed (domain)")

db.commit()

In [4]:
cur.execute("SELECT * FROM parsed LIMIT 1")
cur.fetchall()

[('taplogiq.com',
  'whois',
  0,
  '2022-12-06 09:53:51',
  'GoDaddy.com, LLC',
  146,
  '2018-04-02 18:32:23+00:00',
  '2024-04-02 18:32:23+00:00',
  'ns1.squadhelp.com,ns2.squadhelp.com',
  'abuse@godaddy.com')]

In [9]:
cur.execute("SELECT COUNT(*) FROM parsed")
total_entries = cur.fetchall()[0]

cur.execute("SELECT COUNT(DISTINCT(domain)) FROM parsed")
total_domains = cur.fetchall()[0]

print(f"The database contains {total_entries} entries, over {total_domains} unique domains")

The database contains (156086748,) entries, over (55014675,) unique domains


# Building the missmatch tables

We built one missmatch table per field. 

Each table has the following format :
```sqlite
domain TEXT,
left_id INTEGER,
right_id INTEGER,
left_val <Type>,
right_val <Type>
```
Where `<Type>` is:
- `TEXT` for table `missmatch_registrar`, `missmatch_email`, `missmatch_nameservers`
- `TIMESTAMP` for table `missmatch_update_date`, `missmatch_expiration_date`, `missmatch_registration_date`
- `INTEGER` for table `missmatch_iana_id`

Where `left_id` and `right_id` are the `rowid` of the entry in the `parsed` table 

In [5]:
missmatch_db_path = "../data/artefacts/missmatches.db"
cur.execute(f"ATTACH DATABASE '{missmatch_db_path}' AS missmatch")

In [6]:
cur.execute("CREATE TABLE IF NOT EXISTS missmatch.registrar("
            "domain TEXT,"
            "left_id INTEGER,"
            "right_id INTEGER,"
            "left_val TEXT,"
            "right_val TEXT)")
cur.execute("CREATE TABLE IF NOT EXISTS missmatch.emails("
            "domain TEXT,"
            "left_id INTEGER,"
            "right_id INTEGER,"
            "left_val TEXT,"
            "right_val TEXT)")
cur.execute("CREATE TABLE IF NOT EXISTS missmatch.nameservers("
            "domain TEXT,"
            "left_id INTEGER,"
            "right_id INTEGER,"
            "left_val TEXT,"
            "right_val TEXT)")
cur.execute("CREATE TABLE IF NOT EXISTS missmatch.iana_id("
            "domain TEXT,"
            "left_id INTEGER,"
            "right_id INTEGER,"
            "left_val INTEGER,"
            "right_val INTEGER)")
cur.execute("CREATE TABLE IF NOT EXISTS missmatch.creation_date("
            "domain TEXT,"
            "left_id INTEGER,"
            "right_id INTEGER,"
            "left_val TIMESTAMP,"
            "right_val TIMESTAMP)")
cur.execute("CREATE TABLE IF NOT EXISTS missmatch.expiration_date("
            "domain TEXT,"
            "left_id INTEGER,"
            "right_id INTEGER,"
            "left_val TIMESTAMP,"
            "right_val TIMESTAMP)")
db.commit()

## Computing the different Missmatches

### Registrar

In [7]:
import datetime

start = datetime.datetime.now()
print(start)
cur.execute("INSERT INTO missmatch.registrar "
            "SELECT a.domain, a.rowid, b.rowid, a.registrar, b.registrar "
            "FROM parsed AS a JOIN parsed AS b "
            "ON (a.domain = b.domain) "
            "WHERE (a.rowid < b.rowid) AND (a.registrar != b.registrar)")
db.commit()
print(datetime.datetime.now() - start)

2024-01-25 16:13:22.316987
0:15:25.684484


### Nameservers

In [5]:
import datetime

start = datetime.datetime.utcnow()
print(start)
cur.execute("INSERT INTO missmatch.nameservers "
            "SELECT a.domain, a.rowid, b.rowid, a.nameservers, b.nameservers "
            "FROM parsed AS a JOIN parsed AS b "
            "ON (a.domain = b.domain) "
            "WHERE (a.rowid < b.rowid) AND "
            "(a.nameservers != '') AND "
            "(b.nameservers != '') AND "
            "(a.nameservers != b.nameservers)")
db.commit()
print(datetime.datetime.utcnow() - start)

2024-01-25 11:12:56.369923
0:00:54.678084


### Emails

In [6]:
import datetime

start = datetime.datetime.utcnow()
print(start)
cur.execute("INSERT INTO missmatch.emails "
            "SELECT a.domain, a.rowid, b.rowid, a.emails, b.emails "
            "FROM parsed AS a JOIN parsed AS b "
            "ON (a.domain = b.domain) "
            "WHERE (a.rowid < b.rowid) AND "
            "(a.emails != '') AND "
            "(b.emails != '') AND "
            "(a.emails != b.emails)")
db.commit()
print(datetime.datetime.utcnow() - start)

2024-01-25 11:13:51.054217
0:00:51.249785


### Creation date

In [7]:
import datetime

start = datetime.datetime.now()
print(start)
cur.execute("INSERT INTO missmatch.creation_date "
            "SELECT a.domain, a.rowid, b.rowid, a.creation_date, b.creation_date "
            "FROM parsed AS a JOIN parsed AS b "
            "ON (a.domain = b.domain) "
            "WHERE (a.rowid < b.rowid) AND "
            "(a.creation_date != '') AND "
            "(b.creation_date != '') AND "
            "(a.creation_date != b.creation_date)")
db.commit()
print(datetime.datetime.now() - start)

2024-01-25 12:14:42.309811
0:00:55.791811


### Expiration date

In [8]:
import datetime

start = datetime.datetime.now()
print(start)
cur.execute("INSERT INTO missmatch.expiration_date "
            "SELECT a.domain, a.rowid, b.rowid, a.expiration_date, b.expiration_date "
            "FROM parsed AS a JOIN parsed AS b "
            "ON (a.domain = b.domain) "
            "WHERE (a.rowid < b.rowid) AND "
            "(a.expiration_date != '') AND "
            "(b.expiration_date != '') AND "
            "(a.expiration_date != b.expiration_date)")
db.commit()
print(datetime.datetime.now() - start)

2024-01-25 12:15:38.107503
0:00:55.815163


### IANA IDs

In [9]:
import datetime

start = datetime.datetime.utcnow()
print(start)
cur.execute("INSERT INTO missmatch.iana_id "
            "SELECT a.domain, a.rowid, b.rowid, a.iana_id, b.iana_id "
            "FROM parsed AS a JOIN parsed AS b "
            "ON (a.domain = b.domain) "
            "WHERE (a.rowid < b.rowid) AND (a.iana_id != b.iana_id)")
db.commit()
print(datetime.datetime.utcnow() - start)

2024-01-25 11:16:33.928832
0:00:50.969221


# Cleaning the different tables

## Different names for each registrar

The "Registrar" field of WHOIS and RDAP is not formated, and some registrars have different names used to fill those fields. For example, NameCheap can be found as:
- `NameCheap Ltd`
- `NameCheap Ltd.`
- `Name Cheap LTD`
- `NameCheap.com`

We built a database to match those multiple names to the Cannonical name used in IANA list of registrars with their IANA ID

The database has the following fields:
```sqlite
name TEXT,
cname TEXT,
iana_id INTEGER,
icann_acc INTEGER,
comment TEXT,
recid TEXT,
```

- `name`: the value in the Registrar field
- `cname`: the cannonical name of the registrar
- `iana_id`: the IANA ID of the registrar (positives for official ids, negative for non-official ids that are still wide spread)
- `icann_acc`: if the ID is accredited
- `comment`: additionnal comment
- `recid`: MD5 hash of `name`, for fast lookups

In [10]:
import csv

cur.execute("CREATE TABLE IF NOT EXISTS registrars("
            "name TEXT,"
            "cname TEXT,"
            "iana_id INTEGER,"
            "icann_acc INTEGER,"
            "comment TEXT,"
            "recid TEXT)")

cur.execute("CREATE INDEX IF NOT EXISTS registrars_name ON registrars (name)")


with open("../data/artefacts/registrars.csv") as f:
    entries = csv.DictReader(f)
    cur.executemany("INSERT INTO registrars VALUES(?, ?, ?, ?, ?, ?)", 
                   ((e['name'], e['cname'], e['iana_id'], e['icann_acc'], e['comment'], e['recid'])
                     for e in entries))
db.commit()

In [11]:
cur.execute("SELECT * FROM registrars LIMIT 1")
cur.fetchall()

[('! #1 Host Australia, LLC',
  '! #1 Host Australia, LLC',
  967,
  1,
  'ICANN accredited',
  '708b9e9c851e4d76d44581bcb3476664')]

Updating the `missmatch_regstrar` table to use cannonical names

In [12]:
cur.execute("ALTER TABLE missmatch.registrar ADD COLUMN left_cname TEXT")
cur.execute("ALTER TABLE missmatch.registrar ADD COLUMN right_cname TEXT")
db.commit()

In [14]:
cur.execute("UPDATE missmatch.registrar SET "
            "left_cname = (SELECT registrars.cname FROM registrars WHERE missmatch.registrar.left_val = registrars.name), "
            "right_cname = (SELECT registrars.cname FROM registrars WHERE missmatch.registrar.right_val = registrars.name) "
            "WHERE left_cname IS NULL OR right_cname IS NULL")
db.commit()

In [15]:
cur.execute("SELECT * FROM missmatch.registrar LIMIT 1")
cur.fetchall()

[('seanbeckmedia.com',
  7,
  146,
  'NameCheap, Inc.',
  'NAMECHEAP INC',
  'NameCheap, Inc.',
  'NameCheap, Inc.')]

## Nameserver missmatches for DNS check

First, lets clean false positives

- Parse the nameservers
- Split on ' ' because some have the format `ns1.example.com [OK]` or `ns1.example.com (127.0.0.1)`
- Remove trailing dots
- Put everything to lowercase
- Sort the list, to simplify comparison
- Remove duplicates
- Recompare the entries. Remove false-positives, update missmatches


In [17]:

cur.execute("SELECT rowid, * FROM missmatch.nameservers")

false_positives = []
true_positives = []
while True:
    entries = cur.fetchmany(1_000_000)
    if not entries:
        break
        
    for (rowid, domain, id1, id2, ns1, ns2) in entries:
        ns1_formated = list(set(d.split(' ')[0].strip('.').lower() for d in ns1.split(',')))
        ns2_formated = list(set(d.split(' ')[0].strip('.').lower() for d in ns2.split(',')))

        ns1_formated.sort()
        ns2_formated.sort()

        if ns1_formated == ns2_formated:
            false_positives.append(rowid)
        else :
            true_positives.append((rowid, ns1_formated, ns2_formated))
            
    print(len(false_positives))

print(f"DONE ! {len(false_positives)} to delete, {len(true_positives)} to update")

990251
1980254
2970830
3961423
4950947
5941417
6932003
7210949
DONE ! 7210949 to delete, 71821 to update


In [18]:
cur.executemany("DELETE FROM missmatch.nameservers WHERE rowid=?", ((rid,) for rid in false_positives) )
db.commit()

In [19]:
cur.executemany("UPDATE missmatch.nameservers "
                    "SET left_val=?, right_val=? "
                    "WHERE rowid=?", 
                   ((",".join(ns1), ",".join(ns2), rowid) for (rowid, ns1, ns2) in true_positives))
db.commit()

## Mails

Same as for Nameservers

In [20]:
cur.execute("SELECT rowid, * FROM missmatch.emails")

false_positives = []
true_positives = []
while True:
    entries = cur.fetchmany(1_000_000)
    if not entries:
        break
        
    for (rowid, domain, id1, id2, m1, m2) in entries:
        m1_formated = list(set(d.split(' ')[0].strip('.').lower() for d in m1.split(',')))
        m2_formated = list(set(d.split(' ')[0].strip('.').lower() for d in m2.split(',')))

        m1_formated.sort()
        m2_formated.sort()

        if m1_formated == m2_formated:
            false_positives.append(rowid)
        else :
            true_positives.append((rowid, m1_formated, m2_formated))
            
    print(len(false_positives))

print(f"DONE ! {len(false_positives)} to delete, {len(true_positives)} to update")

15796
31765
49051
61151
DONE ! 61151 to delete, 3824332 to update


In [21]:
cur.executemany("DELETE FROM missmatch.emails WHERE rowid=?", ((rid,) for rid in false_positives) )
db.commit()

In [22]:
cur.executemany("UPDATE missmatch.emails "
                    "SET left_val=?, right_val=? "
                    "WHERE rowid=?", 
                   ((",".join(m1), ",".join(m2), rowid) for (rowid, m1, m2) in true_positives))
db.commit()

## Date missmatches - Round to date

Some entities do not write the time in their dates, so we strip all timestamps to the date, to have a way to compare them

In [23]:
print("Updating creation_date table")
cur.execute("UPDATE missmatch.creation_date "
                "SET left_val = date(left_val), "
                "right_val = date(right_val) ")

print("Deleting entries that are no longer missmatches")
cur.execute("DELETE FROM missmatch.creation_date WHERE left_val = right_val")
db.commit()

Updating creation_date table
Deleting entries that are no longer missmatches


In [25]:
print("Updating expiration_date table")
cur.execute("UPDATE missmatch.expiration_date "
                "SET left_val = date(left_val), "
                "right_val = date(right_val) ")

print("Deleting entries that are no longer missmatches")
cur.execute("DELETE FROM missmatch.expiration_date WHERE left_val = right_val")
db.commit()

Updating expiration_date table
Deleting entries that are no longer missmatches


In [28]:
print("Deleting dates before or equal to Epoch")
cur.execute("DELETE FROM missmatch.creation_date WHERE left_val <= DATE('1970-01-01') OR right_val <= DATE('1970-01-01')")
cur.execute("DELETE FROM missmatch.expiration_date WHERE left_val <= DATE('1970-01-01') OR right_val <= DATE('1970-01-01')")
db.commit()

Deleting dates before or equal to Epoch


# Analysis
## IANA ID missmatches

In [ ]:
cur.execute("SELECT MIN(left_val, right_val), MAX(left_val, right_val), COUNT(DISTINCT(domain)) "
            "FROM missmatch_integer "
            "GROUP BY MIN(left_val, right_val), MAX(left_val, right_val) "
            "ORDER BY COUNT(DISTINCT(domain)) DESC")
invalid_ids = cur.fetchall()

In [13]:
import pandas as pd

invalid_iana = pd.DataFrame(invalid_ids, columns=["id1", "id2", "count"])
invalid_iana

,id1,id2,count
0,1556,1915,96944
1,3951,5555555,3709
2,1556,3824,3067
3,600,735,405
4,1564,2487,235
...,...,...,...
294,1923,3432,1
295,2116,3775,1
296,3768,30149320,1
297,3863,3961,1


In [39]:
from collections import defaultdict

cur.execute("SELECT iana_id, MIN(cname), MIN(comment) FROM registrars GROUP BY iana_id")

iana_ids = pd.DataFrame(cur.fetchall(), columns=["id", "cname", "comment"])
iana_ids.set_index("id", inplace=True)
iana_ids.sort_index()

,cname,comment
id,,
-28,"Open Contact, Ltd",domain.by
-27,APT,emanager.aptg.com.tw
-26,Euronic Oy,euronic.fi
-25,ukfast.net Ltd t/a UKFast.Net Limited,ukfast.net
-24,Loopia AB,Loopia.com
...,...,...
9999,Reserved_by_ICANN_9999,Reserved by ICANN
10007,Domain The Net Technologies Ltd.,IANA-ID Match
10009,Reserved_by_ICANN_10009,Reserved by ICANN


In [40]:
invalid_iana.set_index("id1").join(iana_ids, rsuffix="_1").reset_index(names="id1").set_index("id2").join(iana_ids, rsuffix="_2").sort_values("count", ascending=False).reset_index(names="id2")

,id2,id1,count,cname,comment,cname_2,comment_2
0,1915,1556,96944,"Chengdu West Dimension Digital Technology Co.,...",IANA-ID Match,West263 International Limited,IANA-ID Match
1,5555555,3951,3709,"Webempresa Europa, S.L.",ICANN accredited,NaN,NaN
2,3824,1556,3067,"Chengdu West Dimension Digital Technology Co.,...",IANA-ID Match,Cloud Yuqu LLC,ICANN accredited
3,735,600,405,Rebel Ltd,IANA-ID Match,Rebel.ca Corp.,IANA-ID Match
4,2487,1564,235,TLD Registrar Solutions Ltd.,IANA-ID Match,Internet Domain Service BS Corp,ICANN accredited
...,...,...,...,...,...,...,...
294,1728,433,1,OVH sas,IANA-ID Match,IP Twins SAS,IANA-ID Match
295,1741,303,1,PDR Ltd. d/b/a PublicDomainRegistry.com,IANA-ID Match,Shinjiru Technology Sdn Bhd,IANA-ID Match
296,1795,1441,1,"TurnCommerce, Inc. DBA NameBright.com",ICANN accredited,DropCatch.com 384 LLC,ICANN accredited
297,1807,1441,1,"TurnCommerce, Inc. DBA NameBright.com",ICANN accredited,DropCatch.com 396 LLC,ICANN accredited


Extract the faulty registrars

In [4]:
cur.execute("SELECT iana_id, COUNT(DISTINCT(missmatch.domain)) "
            "FROM missmatch JOIN parsed ON missmatch.left_id = parsed.rowid "
            "WHERE field='nameservers' "
            "GROUP BY iana_id "
            "ORDER BY COUNT(DISTINCT(missmatch.domain)) DESC "
            "LIMIT 100")
ns_faulty_ids = cur.fetchall()

In [10]:
import pandas as pd

ns_missmatches = pd.DataFrame(ns_faulty_ids, columns=["iana_id", "count"])
ns_missmatches[ns_missmatches.iana_id != -1].sum()

iana_id    5688178
count      7085469
dtype: int64

In [ ]:
ns_faulty_ids

In [1]:
%cd ~/app/src

/home/jovyan/app/src


In [2]:
import ns_parser

In [3]:
%cd ~/app/

/home/jovyan/app


In [ ]:
import time

ns_parser.parse_whois_rdap_file("data/ns_rescan_2/whois.json", "data/ns_rescan_2/ns_rescan.db", nb_readers=12)
ns_parser.parse_whois_rdap_file("data/ns_rescan_2/rdap.json", "data/ns_rescan_2/ns_rescan.db", nb_readers=12)
ns_parser.parse_zdns_file("data/ns_rescan_2/zdns.json", "data/ns_rescan_2/ns_rescan.db", nb_readers=12)

Spawning the 12 parsing processes
Spawning the writing process
Sending the file to the parsers...


13601610it [2:56:12, 1481.70it/s]

In [17]:
import sqlite3

db = sqlite3.connect("../data/ns_rescan_2/ns_rescan.db")
cur = db.cursor()

cur.execute("SELECT * FROM missmatch LIMIT 5")
print(cur.fetchall())

[('hinescounsel.com', 7, 287, 'ns-unused-9078.fastdomain.com,ns-unused-9079.fastdomain.com', 'ns1.1marketinghost.com,ns2.1marketinghost.com'), ('hospicandelustraznych.cz', 21, 13452727, 'ns.forpsi.cz,ns.forpsi.it,ns.forpsi.net', 'ns.forpsi.cz (81.2.209.185, 2001:15e8:201:1::d1b9),ns.forpsi.it,ns.forpsi.net'), ('zeiner.no', 55, 13452046, 'ns01.no.brand.one.com,ns02.no.brand.one.com', 'nsno2078h-norid,nsno2079h-norid'), ('etx.studio', 82, 415, 'ns-1031.awsdns-00.org,ns-1636.awsdns-12.co.uk,ns-314.awsdns-39.com,ns-846.awsdns-41.net', 'ns-1031.awsdns-00.org,ns-314.awsdns-39.com,ns-846.awsdns-41.net'), ('sangocongnghiepvietnam.com', 108, 13453689, 'ns1.dotvndns.vn,ns2.dotvndns.vn,nsbak.dotvndns.com', 'ns1.dotvndns.vn,ns2.dotvndns.vndnssec:unsigned,nsbak.dotvndns.com')]


In [13]:
cur.execute("SELECT rowid, * FROM missmatch")

false_positives = []
true_positives = []
while True:
    entries = cur.fetchmany(1_000_000)
    if not entries:
        break
        
    for (rowid, domain, id1, id2, ns1, ns2) in entries:
        ns1_formated = list(set(d.split(' ')[0].strip('.').lower() for d in ns1.split(',')))
        ns2_formated = list(set(d.split(' ')[0].strip('.').lower() for d in ns2.split(',')))

        ns1_formated.sort()
        ns2_formated.sort()

        if ns1_formated == ns2_formated:
            false_positives.append(rowid)
        else :
            true_positives.append((rowid, ns1_formated, ns2_formated))
            
    print(len(false_positives))

print(f"DONE ! {len(false_positives)} to delete, {len(true_positives)} to update")

2345
DONE ! 2345 to delete, 642352 to update


In [18]:
cur.executemany("DELETE FROM missmatch WHERE rowid=?", ((rid,) for rid in false_positives) )
db.commit()

In [19]:
cur.executemany("UPDATE missmatch "
                    "SET left_ns=?, right_ns=? "
                    "WHERE rowid=?", 
                   ((",".join(ns1), ",".join(ns2), rowid) for (rowid, ns1, ns2) in true_positives))
db.commit()